In [ ]:
import xarray as xr
import numpy as np
#from params import *

data_path = "../data/GERICS/"

# Variable overview
# 2D
#   Already fine (i.e., rlat, rlon) : PS, FIB
# 3D
#   Already fine (i.e., rlat, rlon, lev): T
#   rlat, rlon, lev_2: FI
#   rlat, rlon_2, lev: U
#   rlat_2, rlon, lev: V


######################
# Interpolate FI to lev
######################
year = 1986
experiment = "EVAL"

In [ ]:
ds = xr.open_dataset(data_path + experiment + "/FI/FI_" + str(year) + ".nc")
ds_ground = xr.open_dataset(data_path + experiment +  "/FIB/FIB_" + str(year) + ".nc")  # Geopotential at surface

In [ ]:
ds = ds.drop(["hyai", "hybi", "hyam", "hybm"])

In [ ]:
ds_ground = ds_ground.assign_coords({"lev_2": 28.0}).rename({"FIB": "FI"})  # ground is lowermost level

In [ ]:
#ds_ground.rename({"FIB": "FI"})

In [ ]:
ds_combined = xr.concat([ds, ds_ground], dim="lev_2")

In [ ]:
ds_combined.rolling?

In [ ]:
ds_combined = ds_combined.rolling({"lev_2":2}).mean().dropna("lev_2").rename({"lev_2":"lev"})

In [ ]:
ds_combined.lev

In [ ]:
ds_combined = ds_combined.assign_coords({"lev":[float(x) for x in range(1,28)]})

In [ ]:
ds_combined.to_netcdf(
    data_path + experiment + "/FI_interpolated/FI_" + str(year) + ".nc"
)

### Wind speeds

In [ ]:
year = 1986
experiment = "EVAL"

ds_u = xr.open_dataset(data_path + experiment + "/U/U_" + str(year) + ".nc")
ds_v = xr.open_dataset(data_path + experiment + "/V/V_" + str(year) + ".nc")

In [ ]:
# u wind component at grid center is mean of u at its western and eastern margin
ds_u = ds_u.rolling({"rlon_2": 2}).mean()
ds_u = ds_u.rename({"rlon_2": "rlon"}).assign_coords({"rlon": ds_v.rlon})

In [ ]:
# same for v with northern & southern margin
ds_v = ds_v.rolling({"rlat_2": 2}).mean()
ds_v = ds_v.rename({"rlat_2": "rlat"}).assign_coords({"rlat": ds_u.rlat})

In [ ]:
# Compute wind speeds from components
ds = xr.merge([ds_u, ds_v])

In [ ]:
ds

In [ ]:
ds["S"] = (ds.U**2 + ds.V**2)**(1./2)

In [ ]:
ds.S.attrs = {"long_name": "Wind speed", "units": "m/s", "grid_mapping": "rotated_pole"}

In [ ]:
ds.drop(["U", "V"]).to_netcdf(data_path + experiment + "/S/S_" + str(year) + ".nc")